# References
1 https://archive.ics.uci.edu/dataset/124/cmu+face+images


Consider two tasks:

T1: Is this image a picture of Mitchell?

T2: What's the facial expression in the picture?

Your Goal: to report how your network performs at tasks T1 and T2.

In [3]:
# GB: Packages:
# GB: From previous file:
import torch # GB: Contains torch.utils.data?
import torchvision
import torchvision.transforms as transforms
# GB: Added:
import cv2 # For img_loader.
import numpy as np
from torchvision.datasets import ImageFolder # GB: For Helper Function.
# GB: Ref.: https://pytorch.org/vision/main/generated/torchvision.datasets.ImageFolder.html
# GB: Ref.: https://discuss.pytorch.org/t/how-to-load-images-from-different-folders-in-the-same-batch/18942/6
print(torch.__version__) # GB: Added.

2.1.0.post100


In [4]:
# GB: Helper Functions:
def img_loader(filename):
    return cv2.imread(filename, cv2.IMREAD_GRAYSCALE)

def img_folder():
    root = './faces/' # GB: Put in here to not get lost.
    # GB: Changed from 3D to 1D, i.e. from (0.5, 0.5, 0.5) to (0.5,). NB The trailing comma might be important.
    # GB: Parameters: means and standard deviations.
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    return ImageFolder(root=root, loader=img_loader, transform=transform)

dataset = img_folder()
trainset = img_folder()
testset = img_folder()
dummyset = img_folder()
trainset_2 = img_folder()
testset_2 = img_folder()
data = img_folder() # GB: ImageFolder(root=root, loader=img_loader, transform=transform)
print(type(data))
print(data.classes)

# GB: Moved: trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True) # GB: , num_workers=2
# GB: Moved: trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True) # GB: , num_workers=2

# GB: Sequence? It's just a tuple:
# GB: classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
# GB: Alt for faces.userid [1]:
# GB: <userid> is the user id of the person in the image, and this field has 20 values:
classes_1 = ('an2i', 'at33', 'boland', 'bpm', 'ch4f', 'cheyer', 'choon', 'danieln', 'glickman', 'karyadi', 'kawamura', 'kk49', 'megak', 'mitchell', 'night', 'phoebe', 'saavik', 'steffi', 'sz24', 'tammo')
classes_2 = ('angry', 'happy', 'neutral', 'sad')
classes_3 = ('mitchell', 'not_mitchell')
print(classes_1)

<class 'torchvision.datasets.folder.ImageFolder'>
['an2i', 'at33', 'boland', 'bpm', 'ch4f', 'cheyer', 'choon', 'danieln', 'glickman', 'karyadi', 'kawamura', 'kk49', 'megak', 'mitchell', 'night', 'phoebe', 'saavik', 'steffi', 'sz24', 'tammo']
('an2i', 'at33', 'boland', 'bpm', 'ch4f', 'cheyer', 'choon', 'danieln', 'glickman', 'karyadi', 'kawamura', 'kk49', 'megak', 'mitchell', 'night', 'phoebe', 'saavik', 'steffi', 'sz24', 'tammo')


In [5]:
# GB: Data Pump:
# GB: <class 'torchvision.datasets.folder.ImageFolder'>
# GB: From documentation: List[Tuple[str, int]].
# GB: NB Can't use = torch.utils.data.Subset(dataset, indicies) because it drops .imgs and other attributes.
# GB: = dataset.imgs # GB: List of tuples.
# GB: Maybe revisit data types.
# GB: Trying... .imgs:

# GB: Could probably be done outside of NumPy as a list. # GB: Done.
# GB: Assumes all are well formed: # GB: Check .bin images!!!
# GB: Need to fix data types.
fields = ([r[0]
        .replace('_4.pgm', '_4') # GB: Care with order!
        .replace('_2.pgm', '_2')
        .replace('.pgm', '_1')
        .replace('_', '/')
        .replace('./', '') # GB: .
        .split('/') for r in dataset.imgs]) # GB: Chained.

arr = np.column_stack((dataset.imgs, fields))
print('<str> <int> <folder_1> <folder_2> <userid> <pose> <expression> <eyes> <scale>')
print(arr[:5])

# GB: Split out as function?
# GB: Pre-processor:
# GB: <expression> is the facial expression of the person, and this field has 4 values: neutral, happy, sad, angry.
f = 6 # GB: Defaults: 4 (userid), 6 (expression)
fs = np.unique(arr[:, f])
fs = fs.tolist()
print(fs) # GB: ['angry' 'happy' 'neutral' 'sad']
# GB: Double check classes for folder_2 are alphabetical.

dummy = []
for p in range(arr.shape[0]):
    p_value = arr[p, f] # GB: Current value. GB: arr[][]?
    p_class = fs.index(p_value) # GB: Index of current value in values.
    # p_tmp = 0 if p_class == 13 else 1
    tup_dummy = (dummyset.imgs[p][0], int(p_class))
    dummyset.imgs[p] = tup_dummy
    trainset_2.imgs[p] = tup_dummy # GB: Same.
    testset_2.imgs[p] = tup_dummy # GB: Same.
    dummy.append(int(p_class))

# GB: Quick and dirty:
arr = np.column_stack((arr, dummy))
print('<str> <int> <folder_1> <folder_2> <userid> <pose> <expression> <eyes> <scale> <dummy>')
print(arr[:5])

# GB: Resolutions:
# GB: Split out as function?
filter_1 = ((arr[:, 8]=='1') ) # GB: 1 = full res. & (x[:, 5]!='up')
filter_2 = ((arr[:, 8]=='2') ) # GB: 2 = half res
filter_4 = ((arr[:, 8]=='4') ) # GB: 4 = quarter res.
filter_not_1 = ((arr[:, 8]!='1') ) # GB: 1 = full res. | (x[:, 5]=='up')
filter_not_2 = ((arr[:, 8]!='2') ) # GB: 2 = half res.
filter_not_4 = ((arr[:, 8]!='4') ) # GB: 4 = quarter res.

# GB: Parameter:
filter_ = filter_1 # GB: Default: filter_4.
filter_not_ = filter_not_1 # GB: Default: filter_not_4.

arr_filtered = arr[filter_]
arr_filtered_not = arr[filter_not_]

# GB: ...
from sklearn.model_selection import train_test_split
X_y_train, X_y_test = train_test_split(arr_filtered, test_size=0.33, random_state=42, shuffle=False)
# GB: Default test_size: 0.33.

# GB: Remove images that are not in required resolution.
tuple_filtered_not = ('', -1)
tuple_filtered_not_2 = ('', -1)
for p in range(arr_filtered_not.shape[0]):
    tuple_filtered_not = (arr_filtered_not[p][0], int(arr_filtered_not[p][1]))
    dataset.imgs.remove(tuple_filtered_not)
    trainset.imgs.remove(tuple_filtered_not)
    testset.imgs.remove(tuple_filtered_not)
    # GB: Repeat for dummy sets, don't really want two sets.
    tuple_filtered_not_2 = (arr_filtered_not[p][0], int(arr_filtered_not[p][9]))
    dummyset.imgs.remove(tuple_filtered_not_2)
    trainset_2.imgs.remove(tuple_filtered_not_2)
    testset_2.imgs.remove(tuple_filtered_not_2)

# GB: Remove test to leave train.
tuple_test = ('', -1)
tuple_test_2 = ('', -1)
for s in range(X_y_test.shape[0]):
    tuple_test = (X_y_test[s][0], int(X_y_test[s][1]))
    trainset.imgs.remove(tuple_test)
    # GB: Repeat for dummy sets, don't really want two sets.
    tuple_test_2 = (X_y_test[s][0], int(X_y_test[s][9]))
    trainset_2.imgs.remove(tuple_test_2)

# GB: Remove train to leave test.
tuple_train = ('', -1)
tuple_train_2 = ('', -1)
for q in range(X_y_train.shape[0]):
    tuple_train = (X_y_train[q][0], int(X_y_train[q][1]))
    testset.imgs.remove(tuple_train)
    # GB: Repeat for dummy sets, don't really want two sets.
    tuple_train_2 = (X_y_train[q][0], int(X_y_train[q][9]))
    testset_2.imgs.remove(tuple_train_2)

trainloader = torch.utils.data.DataLoader(trainset_2, batch_size=4, shuffle=True, num_workers=0) # GB: Changed num_workers=2 to 0
testloader = torch.utils.data.DataLoader(testset_2, batch_size=4, shuffle=True, num_workers=0) # GB: Changed num_workers=2 to 0
# GB: dataset.imgs.remove((PATH, INDEX))
# GB: Ref.: https://stackoverflow.com/questions/64914820/removing-sample-from-a-pytorch-dataset-object
# GB: If a list is tried instead of a tuple: _
# GB: ValueError: The truth value of an array with more than one element is ambiguous.
# GB: If genuinely not in .imgs: ValueError: list.remove(x): x not in list

<str> <int> <folder_1> <folder_2> <userid> <pose> <expression> <eyes> <scale>
[['./faces/an2i/an2i_left_angry_open.pgm' '0' 'faces' 'an2i' 'an2i'
  'left' 'angry' 'open' '1']
 ['./faces/an2i/an2i_left_angry_open_2.pgm' '0' 'faces' 'an2i' 'an2i'
  'left' 'angry' 'open' '2']
 ['./faces/an2i/an2i_left_angry_open_4.pgm' '0' 'faces' 'an2i' 'an2i'
  'left' 'angry' 'open' '4']
 ['./faces/an2i/an2i_left_angry_sunglasses.pgm' '0' 'faces' 'an2i' 'an2i'
  'left' 'angry' 'sunglasses' '1']
 ['./faces/an2i/an2i_left_angry_sunglasses_2.pgm' '0' 'faces' 'an2i'
  'an2i' 'left' 'angry' 'sunglasses' '2']]
['angry', 'happy', 'neutral', 'sad']
<str> <int> <folder_1> <folder_2> <userid> <pose> <expression> <eyes> <scale> <dummy>
[['./faces/an2i/an2i_left_angry_open.pgm' '0' 'faces' 'an2i' 'an2i'
  'left' 'angry' 'open' '1' '0']
 ['./faces/an2i/an2i_left_angry_open_2.pgm' '0' 'faces' 'an2i' 'an2i'
  'left' 'angry' 'open' '2' '0']
 ['./faces/an2i/an2i_left_angry_open_4.pgm' '0' 'faces' 'an2i' 'an2i'
  'left' 

In [46]:
# GB: "... use the LeNet-5 architecture". Classic or modified?
# GB: Assume modified, what to do about pooling 3x2 or 2x2?
# GB: [1] "<scale> is the scale of the image, and this field has 3 values: 1, 2, and 4.", oh no it doesn't!
# GB: null, _2 or _4.
# GB: Finding 'mitchell' (13, unlucky for some)...
# GB: <expression>...
from __future__ import print_function
import argparse
import torch
import torch.nn as nn # GB: Used in previous.
import torch.nn.functional as F # GB: Used in previous.
import torch.optim as optim
from torchvision import datasets, transforms # GB: Above.

# GB: Parameters:
dim_1 = 29 * 27
dim_2 = 13 * 12
dim_4 = 5 * 4
dim_ = dim_1 # GB: Default: dim_4.

c_1 = 20
c_2 = 4
c_3 = 2
c_ = c_2 # GB: Default: ?

# GB: Forked in start.
class Net(nn.Module):
    def __init__(self, name=None):
        super(Net, self).__init__()
        if name:
            self.name = name
        # GB: Ref.: https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(5, 5))
        # GB: Taken literally, so changed from 3 to 1.
        # GB: ..., which produces ten outputs. Hence, changed from 6 to out_channels=10. Try back to 6...
        # GB: Changed from 5 to kernel_size=(5, 5). Reads as if it necessitate ten outputs, it doesn't necessitate.
        # GB: NB height x width, so assumed 5x5 is height x width, i.e. the same order.
        # GB: Ref.: https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)) # GB: h x w. # GB: Changed from 3x2 to 2x2. 
        # GB: Changed from 2 to kernel_size=(3, 2).
        # GB: NB Height x width, so assumed 3x2 is height x width, i.e. in the same order provided.
        # GB: Step size? Stride changed from 2 to stride=(2, 2), for clarity.
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=16, kernel_size=5)
        # GB: Changed from 6 to in_channels=10. Try back to 6...
        # GB: switched remaining parameters from positional arguments to keyword arguments.
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)) # GB: Added for clarity.# GB: Changed from 3x2 to 2x2.
        # GB: .1 Change the Net.forward method to be able to accept a 28x28 image rather than a 32x32 input.
        self.fc1 = nn.Linear(dim_ * 16, 120) # GB: Changed from 5 * 5 to 8 * 6 to 4 * 3.
        # GB: Try 5 x 4 x 16 for 32 x 30 _4.
        # GB: in_features (int) – size of each input sample = w x h x c.
        # GB: out_features (int) – size of each output sample = 120.
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, c_) # GB: Greater than 10 outputs? Trying exactly 20... ok-ish.
        # GB: Only place to change outputs, apart from metrics at the end.
        
        # compute the total number of parameters
        total_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        print(self.name + ': total params:', total_params)

    def forward(self, x):
        # GB: .2 Change the Net.forward method to be able to accept a 28x28 image rather than a 32x32 input.
        # GB: RuntimeError: shape '[-1, 400]' is invalid for input of size 768
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x))) # GB: Added for clarity.
        # GB: print('x.shape:', x.shape) # GB: Useful check.
        x = x.view(-1, dim_ * 16) # GB: Changed from 5 * 5 to 8 * 6 to 3 * 4, keeping to w x h x c.
        # GB: Try 5 x 4 x 16 for 32 x 30 _4.
        # GB: x = x.view(x.size(0), -1) # GB: Useful check.
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net(name='GavNet') # GB: Keep in main and pass to train/test functions?
print(net) # GB: Added.
# GB: Forked in end.

def train(data_dir, *args, **kwargs): # GB: [1].
# feel free to modify the parameters as appropriate
    # GB: Okay:
    lr, e, m = data_dir['lr'], data_dir['e'], data_dir['m']
    ##########
    # GB: Forked in:
    import torch.optim as optim # GB: To move to top.

    # GB: Added:
    learning_rate = lr #0.002 # GB: Changed from 0.001.
    momentum = m

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=m) # GB: Changed.
    # GB: Ref.: https://pytorch.org/docs/stable/generated/torch.optim.SGD.html
    ##########
    ##########
    # GB: Forked in:
    import time # GB: To move to top.

    start = time.time()
    epochs = e # int(round(e)) #2 # GB: Changed from default of 2.
    resolution = 960 # GB: Added. 
    #an2i_left_angry_open.pgm: indicate a full-resolution image (128 columns by 120 rows); = 15360
    #an2i_left_angry_open_2.pgm: indicates a half-resolution image (64 by 60) and = 3840
    #an2i_left_angry_open_4.pgm: indicates a quarter-resolution image (32 by 30). = 960

    for epoch in range(epochs):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0): # GB: To change from trainloader to data.
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # GB: if inputs.numpy().size != resolution: # GB: Added.
            # GB:   break # GB: Add back in later, if needed.

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics # GB: Adjust for expected data.
            running_loss += loss.item()
            if i % 20 == 19:    # print every 2000 mini-batches # GB: Needs tweeked for dataset. # GB: Chnaged from 2000, 1999
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 20)) # GB: Chnaged from 20.
                running_loss = 0.0

    print('Finished Training')

    end = time.time()
    print('training time ', end-start)
    # GB: Takes approximately ... minutes to complete learning_rate = 0.001.
    ##########

    # GB: pass # GB: Original End.

def test(data_dir, *args, **kwargs): # GB: [1].
    correct = 0
    total = 0
    with torch.no_grad():
        tmp = 0
        for data in testloader:
            tmp += 1 
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('runs of batches', tmp)
    print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))
    print('c', correct)
    # GB: Are there 10,000? No, needs corrected.
    #pass
    return round(1 - (correct / total), 6)

def main():
# Training settings
    # GB: CLI, i.e. not Jupyter? Too much in one exercise.
    # GB: parser.add_argument('--data', type=str, default='./faces', metavar='N'
    # GB:                     , help='Path to directory containing faces dataset.')
    # GB: args = parser.parse_args()
    # GB: data_dir = args.data
    # GB: train(data_dir + '/train') # GB: I didn't take this approach to split data, it should have been in the instructions. 
    # GB: test(data_dir + '/test') # GB: I didn't take this approach to split data, it should have been in the instructions.
    # GB: Put net high enough to be called by all functions?
    data_dir = 'dummy'
    # GB: train(data_dir) # GB: Toggle train.
    test(data_dir)
    path = './models/pytorch-model-2024-05-13.tar' # GB: Changed from .pth to .tar.
    epoch = 10
    # print(net.state_dict())
    torch.save({
            'epoch': epoch,
            'model_state_dict': net.state_dict(),
            # 'optimizer_state_dict': optimizer.state_dict(),
            # 'loss': loss#,
            #...
            }, path) # GB: Changed from PATH.
    # GB: Needs fully refactored to get other variables.
    '''
    # GB: Load:
    model = TheModelClass(*args, **kwargs)
    optimizer = TheOptimizerClass(*args, **kwargs)

    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']

    model.eval()
    # - or -
    model.train()
    '''
    pass

if __name__ == '__main__':
    #main()
    pass
# GB: Stopped here. I've spent days on this exercise trying to understand what the requirements are.
# GB: Raised a ticket, which only confirmed that the instructions are unclear.
# GB: The code above runs a successful model and saves it. I can import in any resolution and move between _
# GB: userid, pose, etc. With more time I could improve performance and accuracy and refactor a more advanced _
# GB: save load operation.

GavNet: total params: 1518260
Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=12528, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=4, bias=True)
)


In [7]:
def run_model(learning_rate):
    data_dir = 'dummy'
    train(data_dir)
    test_result = test(data_dir)
    return test_result

# Python3 code to iterate over a list
list = [0.01, 0.05, 0.1]

# Using for loop
for i in list:
    best = run_model(list)
    print(best)


[1,    20] loss: 1.384
[1,    40] loss: 1.385
[1,    60] loss: 1.399
[1,    80] loss: 1.397
[1,   100] loss: 1.389
[2,    20] loss: 1.387
[2,    40] loss: 1.390
[2,    60] loss: 1.378
[2,    80] loss: 1.388
[2,   100] loss: 1.404
Finished Training
training time  4.0883469581604
runs of batches 52
Accuracy of the network on the 10000 test images: 25 %
52
[1,    20] loss: 1.383
[1,    40] loss: 1.390
[1,    60] loss: 1.388
[1,    80] loss: 1.394
[1,   100] loss: 1.388
[2,    20] loss: 1.386
[2,    40] loss: 1.387
[2,    60] loss: 1.390
[2,    80] loss: 1.389
[2,   100] loss: 1.390
Finished Training
training time  2.8686962127685547
runs of batches 52
Accuracy of the network on the 10000 test images: 25 %
53
[1,    20] loss: 1.385
[1,    40] loss: 1.386
[1,    60] loss: 1.391
[1,    80] loss: 1.386
[1,   100] loss: 1.389
[2,    20] loss: 1.384
[2,    40] loss: 1.382
[2,    60] loss: 1.386
[2,    80] loss: 1.394
[2,   100] loss: 1.389
Finished Training
training time  3.217306137084961
runs

In [49]:
# GB: At this point it had finished running. Make replacement for main(())
# GB: Inspired by [2]
# GB: https://github.com/hyperopt/hyperopt/wiki/FMin
# GB: https://github.com/hyperopt/hyperopt/wiki/FMin#11-the-simplest-case
# GB: Better ref than previous line
# GB: https://hyperopt.github.io/hyperopt/tutorials/02.MultipleParameterTutorial/

# GB: Trying... $ conda install intel::hyperopt # GB: Failed.
# GB: Trying... $ pip install hyperopt # GB: Passed with errors.
# GB: Come back to.

#from hyperopt import hp, tpe, fmin

#def run_model(learning_rate):
    # GB: train(data_dir) # GB: Toggle train.
#    test(data_dir)
#    return loss

# Single line bayesian optimization of polynomial function
#best = fmin(fn=lambda x: run_model(x),
#            space=hp.normal('x', 0.00001, 0.9),
#            algo=tpe.suggest, 
#            max_evals=2000)

from hyperopt import fmin, hp, tpe # GB: In alphabetical order.

def run_model(params):
    train(params)
    test_result = test(params)
    return test_result

def objective(params): # GB: Test function.
    # GB: print(params)
    lr, e = params['lr'], params['e']
    return np.sin(np.sqrt(lr**2 + e**2))

space = {
    'lr': hp.uniform('lr1', 0.001, 0.1), # GB: Default (pytorch): 0.001-.
    'e': hp.uniformint('e', 1, 10),
    'm': hp.uniform('m', 0, 0.9) # GB: Default: 0(pytorch)-0.9.
}

best = fmin(fn=run_model,
            space=space,
            algo=tpe.suggest,
            max_evals=10)

print(best)

[1,    20] loss: 1.089                                                          
[1,    40] loss: 1.152                                                          
[1,    60] loss: 1.164                                                          
[1,    80] loss: 1.211                                                          
[1,   100] loss: 1.255                                                          
[2,    20] loss: 1.046                                                          
[2,    40] loss: 1.383                                                          
[2,    60] loss: 1.169                                                          
[2,    80] loss: 1.235                                                          
[2,   100] loss: 1.180                                                          
Finished Training                                                               
training time                                                                   
3.6575348377227783          

52                                                                              
[1,    20] loss: 1.386                                                          
[1,    40] loss: 1.390                                                          
[1,    60] loss: 1.385                                                          
[1,    80] loss: 1.394                                                          
[1,   100] loss: 1.389                                                          
[2,    20] loss: 1.381                                                          
[2,    40] loss: 1.382                                                          
[2,    60] loss: 1.385                                                          
[2,    80] loss: 1.394                                                          
[2,   100] loss: 1.396                                                          
[3,    20] loss: 1.387                                                          
[3,    40] loss: 1.387      

In [5]:
# GB: Make function to work on either train or test:
# GB: Numerator/Denominator:
class_correct = list(0. for i in range(20)) # GB: Also 20? Changed from 10 to 20, trying...
class_total = list(0. for i in range(20)) # GB: Also 20? Changed from 10 to 20, trying...
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4): # GB: Changed from 4 to , doesn't work past 2. # GB: Batch remainer of 2 not 4
            # GB: Temp fix:
            if len(labels) < 4:
                break
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(4): # GB: Also 20? Changed from 10 to 20, trying...
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / (class_total[i]+0.001))) # GB: quick fix stop div 0
    
# GB: Learning rate 0.001:

# GB: Learning rate 0.1:


NameError: name 'classes' is not defined

In [ ]:
# GB: Make function to work on either train or test:
# GB: Tried with train, not very good on self either?
# GB: Numerator/Denominator:
class_correct = list(0. for i in range(4)) # GB: Also 20? Changed from 10 to 20, trying...
class_total = list(0. for i in range(4)) # GB: Also 20? Changed from 10 to 20, trying...
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4): # GB: Changed from 4 to , doesn't work past 2. # GB: Batch remainer of 2 not 4
            # GB: Temp fix:
            if len(labels) < 4:
                break
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(4): # GB: Also 20? Changed from 10 to 20, trying...
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / (class_total[i]+0.001))) # GB: quick fix stop div 0
    
# GB: Learning rate 0.001:

# GB: Learning rate 0.1:

In [ ]:
# GB: Switches:
# GB: out_channels to in_channels 10 seems to be better than 6.
# GB: Higher resolutions appear to get worse.

# GB: Strategy:
# GB: Poor results on initial class.
# GB: Try 'expressions' a little better but not convincing.
# GB: Try Mitchell vs not Mitchell?

In [ ]:
# GB: There are too many potential dead ends for this to be a useful exercise.
# GB: Also too many unknowns due to lack of instruction.
# GB: It should have kept the original dataset unchanged and made the exercise to wrap the model, not both.
# GB: Changing both simultaniously is likely to cause errors and waste significant amounts of time.
# GB: Too much refactoring!

In [ ]:
# GB: Ref.: https://pytorch.org/tutorials/beginner/saving_loading_models.html
# GB: Save and load models?
# GB: From Ref. and specifically bookmark above, trying...

In [ ]:
# GB: Forked in from Ref.: https://pytorch.org/tutorials/beginner/saving_loading_models.html:

# GB: Changed from TheModelClass to Net.
# GB: Changed from model to net.
# GB: No change to optimizer.

# Define model, see cell above.
# GB: class TheModelClass(nn.Module): # GB: Net.
    # GB: ...

# Initialize model
# GB: model = TheModelClass() # GB: net.

# Initialize optimizer
# GB: optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) # GB: optimimizer.

# Print model's state_dict
print("Model's state_dict:")
for param_tensor in net.state_dict():
    print(param_tensor, "\t", net.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

# GB: Output: # GB: ok.

In [ ]:
# GB: Same Ref.: #saving-loading-model-for-inference
path = './models/pytorch-model-2024-05-13.pth'
torch.save(net.state_dict(), path) # GB: Changed from PATH to path.
# GB: NB Create folder first. ok.

In [ ]:
model = Net(*args, **kwargs) # GB: Changed from TheModelClass to Net. Used model to keep separate form net.
model.load_state_dict(torch.load(path)) # GB: Changed from PATH to path.
model.eval()
# GB: ok-ish.

In [ ]:
# GB: Same Ref.: #save-load-entire-model
torch.save(model, PATH)

In [ ]:
# Model class must be defined somewhere
model = torch.load(PATH)
model.eval()

In [ ]:
# GB: Same Ref.: #export-load-model-in-torchscript-format
# GB: Skipped for now.

In [ ]:
# GB: Same Ref.: #saving-loading-a-general-checkpoint-for-inference-and-or-resuming-training
# GB: Save:
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            ...
            }, PATH)

In [ ]:
# GB: Load:
model = TheModelClass(*args, **kwargs)
optimizer = TheOptimizerClass(*args, **kwargs)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
# - or -
model.train()

In [ ]:
# GB: Same Ref.: #save-load-entire-model
# GB: Assumed that the section before is sufficient, i.e. for a single model.

In [ ]:
# GB: Spare:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5 # unnormalize # GB: Reverse z = (x - mean) / sd.
    npimg = img.numpy() # GB: Converts a tensor object into an numpy.ndarray object.
    # GB: print(npimg) # GB: Added.
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    # GB: In PyTorch, the order of the dimensions is channel*width*height.
    # GB: In Matplotlib, the order of the dimensions is width*height*channel.
    # GB: That’s why the transpose is needed.
    # GB: Ref.: https://discuss.pytorch.org/t/trying-to-understand-torch-transpose/5656
    plt.show()

# get some random training images
# GB: Ref.: https://stackoverflow.com/questions/74289077/attributeerror-multiprocessingdataloaderiter-object-has-no-attribute-next
dataiter = iter(trainset_2) # GB: Changed from trainloader to data to trainset, forked in point.
images, labels = next(dataiter) # GB: Changed from dataiter.next() to data = next(dataiter).
# GB: images = just 1, labels = just 1 in this case.
# GB: Ref.: https://www.w3schools.com/python/ref_func_iter.asp

# show images
imshow(torchvision.utils.make_grid(images))
# GB: images, labels = next(dataiter)
# GB: imshow(torchvision.utils.make_grid(images))

# print labels
# GB: print(' '.join('%5s' % classes[labels[j]] for j in range(4))) # GB: Loader is no longer in batches of 4.
# GB: EDA:
print(len(data)) # GB: Not 640 * 3 (1920).
print(labels) # GB: 
print(classes[labels]) # GB: Seems ok.
print(images.numpy().size) # GB: Resolution(s).
print(images.numpy().shape) # GB: Resolution(s).
# GB: .img attribute?

In [ ]:
# GB: Train set, changed from CIFAR10 to FashionMNIST:
#trainset = torchvision.datasets.FashionMNIST(root='./data/fashion-mnist', train=True,
#                                        download=True, transform=transform)
#trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, #GB: Try 1 instead of 4.
#                                          shuffle=True, num_workers=2)

# GB: Test set, changed from CIFAR10 to FashionMNIST:
#testset = torchvision.datasets.FashionMNIST(root='./data/fashion-mnist', train=False,
#                                       download=True, transform=transform)
#testloader = torch.utils.data.DataLoader(testset, batch_size=4, # GB: Try 1 instead of 4.
#                                         shuffle=False, num_workers=2)

In [ ]:
# GB: Probably not needed:
#trainloader_1 = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)
#testloader_1 = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=True, num_workers=2)
#trainloader_2 = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)
#testloader_2 = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=True, num_workers=2)
#trainloader_4 = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)
#testloader_4 = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=True, num_workers=2)

In [ ]:
#mask = np.isin(element, test_elements, invert=True)
#arr = np.delete(arr, indexes, axis=0)
# GB: Alt option by index m = (arr[:, None] != X_y_train).all(-1).any(1) # Filter A that are present in B
# Alt option x is your dataset
#x = numpy.random.rand(100, 5)
#indices = numpy.random.permutation(x.shape[0])
#training_idx, test_idx = indices[:80], indices[80:]
#training, test = x[training_idx,:], x[test_idx,:]
# Alt, alt , option
# construct the full dataset
#dataset = ImageFolder("image-folders",...)
# select the indices of all other folders
#idx = [i for i in range(len(dataset)) if dataset.imgs[i][1] != dataset.class_to_idx['class_s']]

In [ ]:
# GB: class Net(nn.Module):
# GB:    def __init__(self):
# GB:       pass # GB: The pass statement is used as a placeholder for future code.
# GB:
# GB:    def forward(self, x):
# GB:        pass # GB: The pass statement is used as a placeholder for future code.

## References:
1. https://www.geeksforgeeks.org/args-kwargs-python/
1. https://discuss.pytorch.org/t/hyperparameter-tuning-using-bayesian-optimization/36145/2